# GMSO - General Molecular Simulation Object

## Complex Atomtyping

[GMSO](https://github.com/mosdef-hub/gmso) allows for engine agnostic python object to store everything needed for writing molecule input files for simulation.


This notebook is designed to provide basics for interfacing with GMSO. Tutorials get more in-depth in terms of complex usage denoted by the `beginners`, `intermediates`, and `experts` tags on these notebooks. For a full list of the examples discussed in these tutorials, please see [the GMSO Tutorials README](README.md).

After completing this notebook, move on to [this notebook](04_for_intermediates.ipynb) to learn about how to write complex coarse-grained systems in `GMSO`.

Contained in this notebook is examples of:
* Typed vs untyped topologies
* Interactive `networkx`
* More complex writing

---

In [ ]:
import gmso
from gmso.parameterization import apply

## Typed vs untyped topologies
---

In [40]:
top = gmso.Topology.load("source_files/ethane.json")
top.is_typed()

False

In [41]:
ff = gmso.ForceField("source_files/ethane.xml")
ptop = apply(top, ff, identify_connections=True)
ptop.is_typed()

/Users/calcraven/Dropbox/Mac/Documents/Vanderbilt/Research/MoSDeF/gmso/gmso/core/forcefield.py:398: UserWarning: AngleType between atoms CT, CT and HC is missing from the ForceField
  warnings.warn(msg)
/Users/calcraven/Dropbox/Mac/Documents/Vanderbilt/Research/MoSDeF/gmso/gmso/core/forcefield.py:398: UserWarning: AngleType between atoms HC, CT and HC is missing from the ForceField
  warnings.warn(msg)
/Users/calcraven/Dropbox/Mac/Documents/Vanderbilt/Research/MoSDeF/gmso/gmso/core/forcefield.py:459: UserWarning: DihedralType between atoms HC, CT, CT and HC is missing from the ForceField.
  warnings.warn(msg)
/Users/calcraven/Dropbox/Mac/Documents/Vanderbilt/Research/MoSDeF/gmso/gmso/core/topology.py:818: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  all_scales[index][scaling_interaction_idxes[interaction]] = va

True

## Interactive networkx
---

In [19]:
top = gmso.Topology.load("source_files/ethane.json")
top.is_typed()
ff = gmso.ForceField("source_files/ethane.xml")
ptop = apply(
    top, ff, identify_connections=True, 
    ignore_params=["site", "bond", "angle", "dihedral", "improper"],
    remove_untyped=False
)

/Users/calcraven/Dropbox/Mac/Documents/Vanderbilt/Research/MoSDeF/gmso/gmso/core/forcefield.py:367: UserWarning: BondType between atoms CT and CT is missing from the ForceField
  warnings.warn(msg)
/Users/calcraven/Dropbox/Mac/Documents/Vanderbilt/Research/MoSDeF/gmso/gmso/core/forcefield.py:367: UserWarning: BondType between atoms CT and HC is missing from the ForceField
  warnings.warn(msg)
/Users/calcraven/Dropbox/Mac/Documents/Vanderbilt/Research/MoSDeF/gmso/gmso/core/forcefield.py:398: UserWarning: AngleType between atoms CT, CT and HC is missing from the ForceField
  warnings.warn(msg)
/Users/calcraven/Dropbox/Mac/Documents/Vanderbilt/Research/MoSDeF/gmso/gmso/core/forcefield.py:398: UserWarning: AngleType between atoms HC, CT and HC is missing from the ForceField
  warnings.warn(msg)
/Users/calcraven/Dropbox/Mac/Documents/Vanderbilt/Research/MoSDeF/gmso/gmso/core/forcefield.py:459: UserWarning: DihedralType between atoms HC, CT, CT and HC is missing from the ForceField.
  warnin

In [9]:
from gmso.formats.networkx import interactive_networkx_atomtypes
interactive_networkx_atomtypes(ptop)

/Users/calcraven/miniconda3/envs/gmso-dev/lib/python3.11/site-packages/ipywidgets/widgets/interaction.py:43: DeprecationWarning: `ipykernel.pylab.backend_inline` is deprecated, directly use `matplotlib_inline.backend_inline`
  from ipykernel.pylab.backend_inline import flush_figures


interactive(children=(Dropdown(description='Label', options=('atom_type.name', 'charge', 'mass', 'element', 'l…

## More complex writing
---

In [24]:
import mbuild as mb
from pdbfixer import PDBFixer 
from openmm.app import PDBFile

from gmso.external import from_mbuild, from_parmed
! wget -O source_files/3O49.pdb https://files.rcsb.org/view/3O49.pdb

fixer = PDBFixer(filename="source_files/3O49.pdb")
fixer.addMissingHydrogens(7.0)
PDBFile.writeFile(fixer.topology, fixer.positions, open("saved_files/protein.pdb", 'w'))
protein = mb.load("saved_files/protein.pdb")
children_to_remove = set()
for particle in protein.particles():
    if "HOH" in particle.parent.name:
        children_to_remove.add(particle)
protein.remove(children_to_remove)

water = mb.load("O", smiles=True)
water.name="water"
cpd = mb.fill_box([protein, water], [1,1000], box=[10,20,10])


top = cpd.to_gmso()
top

--2023-12-20 20:08:35--  https://files.rcsb.org/view/3O49.pdb
Resolving files.rcsb.org (files.rcsb.org)... 128.6.159.245
Connecting to files.rcsb.org (files.rcsb.org)|128.6.159.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/plain]
Saving to: ‘source_files/3O49.pdb’

source_files/3O49.p     [ <=>                ] 119.76K  --.-KB/s    in 0.09s   

2023-12-20 20:08:36 (1.25 MB/s) - ‘source_files/3O49.pdb’ saved [122634]



<Topology Topology, 4887 sites,
 3900 connections,
 0 potentials,
 id: 7216980112>

In [25]:
waterXml = gmso.ForceField("source_files/tip3p.xml")
waterXml.combining_rule = "geometric"
proteinXml = gmso.ForceField("oplsaa")
proteinXml = gmso.ForceField("../../../gmso/gmso/utils/files/gmso_xmls/test_ffstyles/gaff.xml")

molecules = top.unique_site_labels(name_only=True)
ffDict = {}
for mol in molecules:
    if mol == "water":
        ffDict[mol] = waterXml
    else:
        ffDict[mol] = proteinXml

parameterizedTop = apply(
    top, ffDict, identify_connections=True,
    ignore_params=["site", "bond", "angle", "dihedral", "improper"],
    remove_untyped=False,
)

/Users/calcraven/miniconda3/envs/gmso-dev/lib/python3.11/site-packages/pydantic/v1/validators.py:157: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(v)
/Users/calcraven/miniconda3/envs/gmso-dev/lib/python3.11/site-packages/pydantic/v1/validators.py:157: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(v)
/Users/calcraven/miniconda3/envs/gmso-dev/lib/python3.11/site-packages/pydantic/v1/validators.py:157: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.

/Users/calcraven/Dropbox/Mac/Documents/Vanderbilt/Research/MoSDeF/gmso/gmso/core/topology.py:818: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  all_scales[index][scaling_interaction_idxes[interaction]] = value
/Users/calcraven/Dropbox/Mac/Documents/Vanderbilt/Research/MoSDeF/gmso/gmso/core/topology.py:818: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  all_scales[index][scaling_interaction_idxes[interaction]] = value
/Users/calcraven/Dropbox/Mac/Documents/Vanderbilt/Research/MoSDeF/gmso/gmso/core/topology.py:818: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract

In [31]:
parameterizedTop.is_fully_typed()

False

In [34]:
missing_types = parameterizedTop.get_untyped("topology")
for missing in missing_types:
    print(f"Missing types for {missing}: {len(missing_types[missing])}")

Missing types for sites: 0
Missing types for bonds: 0
Missing types for angles: 7
Missing types for dihedrals: 108
Missing types for impropers: 1552
